In [2]:
from dynamicme.dynamic import DynamicME

/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/solvers/__init__.py:63: UserWarning: No LP solvers found
  warn("No LP solvers found")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


## Load the built ME model

In [3]:
from cobrame.io.json import load_reduced_json_me_model, load_json_me_model
import pickle

In [4]:
with open('FoldME_Ali_keff.pickle','rb') as f:
    me = pickle.load(f)

# Set up the dynamic simulation

## Here, we will simulate the complex medium originally investigated by [Beg et al., 2007](http://www.pnas.org/content/104/31/12663)

### Metabolites to track

In [5]:
tracked_mets = [
    me.metabolites.ac_p,
    me.metabolites.gal_p,
    me.metabolites.glc__D_p,
    me.metabolites.glyc_p,
    me.metabolites.lac__L_p, 
    me.metabolites.malt_p
    ]
extra_rxns_tracked = [me.reactions.biomass_dilution, me.reactions.EX_o2_e]

### Also track translation fluxes

In [14]:
me.reactions.query('biomass')

[<SummaryVariable biomass_dilution at 0x7ac3ae9bf7f0>,
 <SummaryVariable protein_biomass_to_biomass at 0x7ac3aeb724a8>,
 <SummaryVariable mRNA_biomass_to_biomass at 0x7ac3ae975160>,
 <SummaryVariable tRNA_biomass_to_biomass at 0x7ac3ae9750f0>,
 <SummaryVariable rRNA_biomass_to_biomass at 0x7ac3ae975080>,
 <SummaryVariable ncRNA_biomass_to_biomass at 0x7ac3ae975390>,
 <SummaryVariable DNA_biomass_to_biomass at 0x7ac3ae975400>,
 <SummaryVariable lipid_biomass_to_biomass at 0x7ac3ae975470>,
 <SummaryVariable constituent_biomass_to_biomass at 0x7ac3ae9754e0>,
 <SummaryVariable prosthetic_group_biomass_to_biomass at 0x7ac3ae975550>,
 <SummaryVariable peptidoglycan_biomass_to_biomass at 0x7ac3ae9755c0>,
 <SummaryVariable biomass_constituent_demand at 0x7ac348577940>]

In [15]:
rxns_trsl = me.reactions.query('translation_')
print(len(rxns_trsl))
extra_rxns_tracked = extra_rxns_tracked + rxns_trsl

rxns_biomass = me.reactions.query('biomass')
print(len(rxns_biomass))
extra_rxns_tracked = extra_rxns_tracked + rxns_biomass


rxns_cplx_formation = me.reactions.query('formation_')
print(len(rxns_cplx_formation))
extra_rxns_tracked = extra_rxns_tracked + rxns_cplx_formation


from cobrame.core.reaction import MetabolicReaction
import pandas as pd

rows_tracked = []

for met_track in tracked_mets:
    mid_c = met_track.id.replace('_p','_c')
    mid_e = met_track.id.replace('_p','_e')
    met_c = me.metabolites.get_by_id(mid_c)
    met_e = me.metabolites.get_by_id(mid_e)
    for rxn in met_track.reactions:
        if isinstance(rxn, MetabolicReaction) and rxn.keff and met_c in rxn.metabolites or met_e in rxn.metabolites:
            extra_rxns_tracked.append(rxn)
            rows_tracked.append({'met':met_track.id, 'rxn':rxn.id})
            #print met_track,'\t', rxn.id, '\t', rxn.keff          
            
df_tracked = pd.DataFrame(rows_tracked)

1579
12
1448


In [17]:
dyme = DynamicME(me)

T = 10   # hours
V = 1.   # L
X0 = 0.00675 / V  # g/L
c0_dict = {'glc__D_e': 0.4,
           'lac__L_e': 0.4,
           'malt_e': 0.4,
           'gal_e': 0.4,
           'glyc_e':0.4,
           'ac_e': 0.0}
### Convert from g/L to mmol
for mid,c in c0_dict.items():
    met = me.metabolites.get_by_id(mid)
    c0_dict[met.id] = c / met.formula_weight * 1000

### Set max uptake rate
LB_EX = -10.
LB_O2 = -20.

lb_dict={}
ub_dict={}

for mid in c0_dict.keys():    
    rxn = dyme.get_exchange_rxn(mid)    
    if rxn.id == 'EX_o2_e':
        lb = LB_O2
    else:
        lb = LB_EX
    #rxn.lower_bound = lb
    lb_dict[rxn.id] = lb

#me.reactions.EX_o2_e.lower_bound = LB_O2
lb_dict[me.reactions.EX_o2_e.id] = LB_O2

print(c0_dict)
print(lb_dict)
print(ub_dict)

sim_params = {
    'T':T,
    'X0':X0,
    'c0_dict':c0_dict,
    'lb_dict':lb_dict,
    'ub_dict':ub_dict,
    'extra_rxns_tracked':extra_rxns_tracked,
    'ZERO_CONC':0.
}

{'glc__D_e': 2.220299442904667, 'lac__L_e': 4.4908498933423155, 'malt_e': 1.1685776026677226, 'gal_e': 2.220299442904667, 'glyc_e': 4.34339676647141, 'ac_e': 0.0}
{'EX_glc__D_e': -10.0, 'EX_lac__L_e': -10.0, 'EX_malt_e': -10.0, 'EX_gal_e': -10.0, 'EX_glyc_e': -10.0, 'EX_ac_e': -10.0, 'EX_o2_e': -20.0}
{}


In [18]:
growth_rxn = me.reactions.biomass_dilution

# Perform DynamicME simulation

In [19]:
dyme = DynamicME(me)

In [20]:
import time

tic = time.time()

result = dyme.simulate_batch(T = 0.2, # testing
                             c0_dict=c0_dict,
                             X0=X0, 
                             prec_bs=1e-3,
                             ZERO_CONC=0.,
                             extra_rxns_tracked=extra_rxns_tracked,
                             lb_dict={}, 
                             verbosity=1,
                            cplx_conc_dict0 = {data.complex.id: 0.0 for data in me.complex_data}
)   
toc = time.time()-tic

Test to get cplx concs failed! Exiting now.


Exception: 'NoneType' object is not subscriptable

In [ ]:
print('%g seconds'%toc)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def fractions_from_result(result):
    """
    Build a tidy dataframe of relative mass fractions straight from
    result['complex'] (which is mmol/gDW per complex). We normalize by the
    sum at each time point so it's a fraction (unitless).
    """
    rows = []
    times = result.get("time", [])
    complexes_list = result.get("complex", [])
    for t, cdict in zip(times, complexes_list):
        cdict = cdict or {}
        # ignore non-finite values
        values = np.array(list(cdict.values()), dtype=float)
        total = np.nansum(values)
        if not np.isfinite(total) or total <= 0:
            continue
        for cid, conc in cdict.items():
            conc = float(conc) if conc is not None else np.nan
            if not np.isfinite(conc) or conc <= 0:
                continue
            rows.append({
                "time": float(t),
                "complex_id": cid,
                "fraction": conc / total
            })
    return pd.DataFrame(rows)

df_mass = fractions_from_result(result)

if df_mass.empty:
    print("No fraction data could be computed (df_mass is empty). "
          "Check that result['complex'] contains nonzero concentrations.")
else:
    # Top-N complexes by fraction at the final time
    final_time = df_mass["time"].max()
    topN = 10
    top_ids = (
        df_mass[df_mass["time"] == final_time]
        .sort_values("fraction", ascending=False)
        .head(topN)["complex_id"]
        .tolist()
    )

    # Line plot of top-N
    plt.figure(figsize=(10, 6))
    for cid in top_ids:
        sub = df_mass[df_mass["complex_id"] == cid].sort_values("time")
        plt.plot(sub["time"], sub["fraction"], label=cid)
    plt.xlabel("Time (h)")
    plt.ylabel("Relative fraction (unitless)")
    plt.title(f"Top {topN} complexes by final relative abundance")
    plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", fontsize=8)
    plt.tight_layout()
    plt.show()

    # Stacked area (all complexes) — can be busy if there are many
    pivot = df_mass.pivot_table(index="time", columns="complex_id",
                                values="fraction", fill_value=0.0)
    pivot = pivot.sort_index()
    ax = pivot.plot.area(figsize=(12, 6), legend=False)
    ax.set_xlabel("Time (h)")
    ax.set_ylabel("Fraction of total proteome")
    ax.set_title("Proteome composition over time (relative)")
    plt.tight_layout()
    plt.show()


In [ ]:
result['biomass']


In [ ]:
result['rxn_flux']


# Convert solution's metabolite concentrations from mM to g/L

In [ ]:
from dynamicme.dynamic import ParamOpt

popt = ParamOpt(me, sim_params, growth_rxn=growth_rxn.id)
sol = popt.compute_conc_profile(result)

In [ ]:
df_mw = pd.DataFrame([{'id':m.id,'mass':m.mass if hasattr(m,'mass') else m.formula_weight,'name':m.name} for m in me.metabolites])

sol_gL = sol.copy()
variables = c0_dict.keys()

for col in sol.columns:
    if col in variables:
        c_mM = sol[col]
        mw = df_mw[ df_mw.id==col].mass.values[0]
        c_gL = c_mM * mw / 1000.
        sol_gL[col] = c_gL
        
df_ex = pd.DataFrame(result['ex_flux'])

# Plot metabolite concentration profiles

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

dsplot = sol_gL[[c for c in sol_gL.columns if c != 'biomass' and 'EX_' not in c]].melt(id_vars='time')

dsplot.variable.unique()

palette = ['#660066','#00FF00','#000000','#FF00FF','#0000FF','#FF0000']
sns.set_palette(palette)
g = sns.FacetGrid(dsplot, hue='variable', size=5)
g.map(plt.plot, 'time','value')
g.add_legend()

In [ ]:
sol_gL